# Note book de test de l'utilisation de la library HdfsCli
## Realisation d'opérations de base 
- creation de repertoires
- liste (ls) des fichiers et repertoires HDFS
- lecture des carateristiques de fichiers
- upload de fichiers vers HDFS
- download de fichiers depuis HDFS
- suppression de fichiers et de repertoires

## Resolution @ip des datanodes
A noter qu'il est necéssaire que la machine qui execute le notebook jupyter doit avoir la resolution des @ip des datanodes du cluster hdfs
Cela peut etre fait simplement dans le /etc/hosts ; par exemple:
51.75.248.35 datanode
51.75.248.35 datanode-public
51.75.248.35 datanode1
51.75.248.35 datanode2

## Sandbox docker d'un cluster HDFS 
https://github.com/pevn14/hadoop-hdfs-sandbox

In [1]:
from hdfs import InsecureClient, HdfsError

In [2]:
# server = 'http://192.168.1.31:9870'
server = 'http://vps-cf21ed4b.vps.ovh.net:9870'
# ne pas oublier de declarer les ip des datanodes dans /etc/hosts
client = InsecureClient(server, user='hadoop')

In [3]:
# Lister les repertoires et fichiers dans la racine HDFS
files = client.list('/')
print(files)

[]


In [22]:
# supprimer le repertoire de test HDFS ; pour demarrer sur config de test propre
file = '/test-client-hdfs'
client.delete(file, recursive=True)  # flag recursive car repertoire non vide

True

In [23]:
# creer un repertoire pour les tests 
client.makedirs('/test-client-hdfs')

In [7]:
# Copier un fichier local vers HDFS
file = '/home/patrick/working/hadoop-hdfs-sandbox/lorem-ipsum-generator/lorem_ipsum_10k_lines.txt'
try:
    ret = client.upload('/test-client-hdfs', file, overwrite=True)  # avec le flag overwrite
except HdfsError as e:
    print(f"Une erreur s'est produite lors de l'upload : {e}")    

In [9]:
# Copier un fichier local de 10 million de lignes vers HDFS
# prend quelques minutes
file = '/home/patrick/working/hadoop-hdfs-sandbox/lorem-ipsum-generator/lorem_ipsum_10m_lines.txt'
try:
    ret = client.upload('/test-client-hdfs', file) # sans le flag overwrite
except HdfsError as e:
    print(f"Une erreur s'est produite lors de l'upload : {e}")  # tentez l'overwrite pour declencher l'erreur 

Une erreur s'est produite lors de l'upload : Remote path '/test-client-hdfs/lorem_ipsum_10m_lines.txt' already exists.


In [10]:
# Copier un fichier local vers HDFS en une erreur
file = 'un-fichier-qui-n-existe-pas.txt'
try:
    ret = client.upload('/test-client-hdfs', file, overwrite=True)  # avec le flag overwrite
except HdfsError as e:
    print(f"Une erreur s'est produite lors de l'upload : {e}")    

Une erreur s'est produite lors de l'upload : Local path 'un-fichier-qui-n-existe-pas.txt' does not exist.


In [11]:
# Lister les fichiers du repertoire de test
files = client.list('/test-client-hdfs')
print(files)

['lorem_ipsum_10k_lines.txt', 'lorem_ipsum_10m_lines.txt']


In [24]:
# Upload de fichiers vers HDFS en mode ligne par ligne
# Chemin du fichier local
local_path = '/home/patrick/working/hadoop-hdfs-sandbox/lorem-ipsum-generator/lorem_ipsum_100k_lines.txt'

# Chemin du fichier dans HDFS
hdfs_path = '/test-client-hdfs/file.txt'

try:
    with open(local_path, 'r', encoding='utf-8') as reader, client.write(hdfs_path, encoding='utf-8') as writer:
        for line in reader:
            writer.write(line)
except HdfsError as e:
    print(f"Une erreur s'est produite lors de l'upload : {e}")  # testez l'erreur avec une tentative d'overwrite

In [15]:
# Lister les fichiers du répertoire hdsf
files = client.list('/test-client-hdfs')
print(files)

['file.txt', 'lorem_ipsum_10k_lines.txt', 'lorem_ipsum_10m_lines.txt']


In [16]:
# Retrieving a file or folder status and content summary
file = '/test-client-hdfs/lorem_ipsum_10m_lines.txt'
status = client.status(file)
content = client.content(file)
print(status)
print(content)

{'accessTime': 1709996149379, 'blockSize': 134217728, 'childrenNum': 0, 'fileId': 16408, 'group': 'supergroup', 'length': 936397436, 'modificationTime': 1709996229303, 'owner': 'hadoop', 'pathSuffix': '', 'permission': '644', 'replication': 3, 'storagePolicy': 0, 'type': 'FILE'}
{'directoryCount': 0, 'ecPolicy': 'Replicated', 'fileCount': 1, 'length': 936397436, 'quota': -1, 'snapshotDirectoryCount': 0, 'snapshotFileCount': 0, 'snapshotLength': 0, 'snapshotSpaceConsumed': 0, 'spaceConsumed': 2809192308, 'spaceQuota': -1, 'typeQuota': {}}


In [17]:
# Copier des fichiers depuis HDFS vers local
file = '/test-client-hdfs/lorem_ipsum_10k_lines.txt'
client.download(file, 'file_download', overwrite=True)

'/home/patrick/working/hadoop-hdfs-sandbox/python-client/file_download'

In [18]:
# supprimer un fichier du repertoire HDFS
file = '/test-client-hdfs/lorem_ipsum_10k_lines.txt'
client.delete(file)

True

In [19]:
# netoyage du repertoire de test HDFS
file = '/test-client-hdfs'
client.delete(file, recursive=True)  # flag recursive car repertoire non vide

True